<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
    Introduction to HTTP using Python
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:56%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# HTTP Servers

This section will somewhat parallel the prior one.  We look at the very simple server in the Python standard library, then move up to the widely use third-party library Flask. 

The nomenclature of Python web servers can be somewhat confusing.  Actual **servers** focus on the underlying sockets, i.e. servicing connections efficiently and in a scalable way.  However, the actual processing of HTTP headers and bodies, both in receiving requests and generating responses, is usually passed to an HTTP **framework**.  These different bits of code usually communicate over an interface called WSGI (Web Services Gateway Interface).

The nomenclature is fuzzy, however.  "Servers" such as Gunicorn, Nginx, Tornado, Twisted, or indeed the well-established Apache, all themselves include the ability to serve basic pages with no framework, or in a sense include a basic framework.  "Frameworks" such as Flask, Django, web2py, or TurboGears (or *many others*) also include a basic development server in their package.

At opposite ends of the spectrum of frameworks are Django and Flask.  The others fall somewhere in the middle. Django includes everything you can imagine, and many things you cannot, related to *web development*, from a MVC (model-view-controller) architecture, to its own database object-relational mapper (ORM), to an elaborate templating system, to an integrated administration interface, to an embedded framework of metaclasses, to many other elements.

Flask, in contrast, does as little as possible, in as simple a way as possible.  If you wish to do those many other web development tasks, it does not get in the way of you using other tools for each task, but it does not prescribe particular choices, nor include them in the library itself.

A week long bootcamp might scratch the surface on *some* of the corners of Django; an hour of video presentation in this course (perhaps with a couple more hours in the related course, *Secure RESTful APIs using Python*) will cover most of what Flask does.

## The Standard Library

Python's standard library contains a simple HTTP server called `http.server`.  If all you wish to do is match the original intention of the HTTP protocol—i.e. make local resources navigable and available remotely—this server does that.  For illustration, we use the Jupyter `HTML` and `Image` display objects to render content, but we could look at the raw bodies if we preferred.

In order to get this server running, I simply ran the following in the local directory:

```bash
python -m http.server 2503
```

This generates and HTML version of the directory listing by the server.

In [ ]:
from IPython.core.display import HTML, Image
import requests

resp = requests.get('http://popbox.kdm.local:2503')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

HTML(resp.text)

This also provides paths to those files that are present, served over HTTP.  Content type is inferred from extension.

In [ ]:
resp = requests.get('http://popbox.kdm.local:2503/greetings.txt')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

print()
print(resp.content.decode()[:75])

In [ ]:
resp = requests.get('http://popbox.kdm.local:2503/no-such-file.txt')
print(resp.status_code, resp.reason)
print()
print(resp.text)

The Jupyter widget properly renders pages, including referenced resources.

In [ ]:
resp = requests.get('http://popbox.kdm.local:2503/hello.html')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

HTML(resp.text)

In [ ]:
print(resp.text)

---

In [ ]:
resp = requests.get('http://popbox.kdm.local:2503/psf-logo.png')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')
print()

Image(resp.content)

---

Of course, quite likely you might like a server to do something beyond sharing files on disk.  This is possible in `http.server` by defining a class with methods `.do_GET()`, `do_POST()` and so on.  Let's first look at an interaction with a customized server:

In [ ]:
resp = requests.get('http://popbox.kdm.local:2504/some-file.txt')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

HTML(resp.text)

---

This server simply indicates the path requested by the client.  It is free to do what it likes with that information, or indeed with any of the header information provided.  The subclass `SimpleHTTPRequestHandler` was used when we simply ran the module itsef. It decided to examine those HTTP paths, and return 404 if they do not correspond local file, but otherwise return the file content with a header indicating an appropriate `Content-Type`.

A server we write ourselves may not take paths as corresponding to files, but it would be free to check the filesystem if that is relevant.  Alternately, it might perform calculations, query databases, or take other actions appropriate to the path and/or query parameters provided in the request.

The code that makes up this server is not complex, and it illustrates how you might customize it.  However, the style of coding is distinctly "old fashioned", as will become more evident in comparison to `requests`.

```python
#!/usr/bin/env python
from http.server import BaseHTTPRequestHandler, HTTPServer
from time import asctime

class Server(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header('X-INE-Course', 'HTTP using Python')
        self.end_headers()
        self.wfile.write(f"<h3>Request details:</h3>\n".encode())
        self.wfile.write(f"<p>You accessed path: {self.path}</p>\n".encode())
        self.wfile.write(b"<blockquote><pre>\n")
        self.wfile.write(f"{self.requestline}\n{self.headers}".encode())
        self.wfile.write(b"</pre></blockquote>\n")

host, port = "0.0.0.0", 2504
server = HTTPServer((host, port), Server)

print(f"{asctime()} Server Start: ({host}:{port})")
try:
    server.serve_forever()
except KeyboardInterrupt:
    server.server_close()
print(f"{asctime()} Server Stop")
```

When run at the command line, we can cancel the server with Ctrl-C.  For example:

```
% ./server4.py
Thu Jun 10 21:21:01 2021 Server Start: (0.0.0.0:2504)
192.168.50.11 - - [10/Jun/2021 21:21:03] "GET / HTTP/1.1" 200 -
192.168.50.11 - - [10/Jun/2021 21:21:31] "GET /some-file.txt HTTP/1.1" 200 -
^CThu Jun 10 21:23:03 2021 Server Stop
```

## Modern HTTP with Flask

Flask provides a flexible, robust, and modern API for writing HTTP servers. It cooperates easily with high-performance web servers, when desired.  The examples in this course will use the "Werkzeug" development server that is included in Flask. However, the exact same framework code can be used for different underlying servers.

For example, I might launch one of the servers provided in the repository for this course with, e.g.:

```bash
$ FLASK_APP=myserver.py flask run
```

In practice, I include a main block in the files to launch them as executable files, as you will see.  But if I configured a production system, I might instead launch the very same server using, e.g.:

```bash
$ gunicorn -w 4 -b 127.0.0.1:2510 myserver:app
$ uwsgi --http 127.0.0.1:2511 --module myserver:app
$ twistd -n web --port tcp:2512 --wsgi myserver.app
```

The command-line switches are slightly different, but those lines would launch the same web application, `myserver.py` on ports 2501, 2511, 2512, served by different servers.  Other servers are similar as well; some may require a half dozen, or fewer, lines to launch the identical Flask application rather than only a command line.



## Example servers

Let's take a look at some servers implemented for this course first; then we will look at a few other capabilities.  In the first few lessons, we used `server1.py` to provide a `/greeting` path.  That was done with the following:

```python
from flask import Flask, request, make_response
app = Flask(__name__)

@app.route('/greeting')
def greeting():
    lang = request.args.get('lang', 'en')
    greet = {'en': 'Hello', 'zh': 'Nǐn hǎo', 'fr': 'Bonjour'}[lang]
    who = request.headers.get('X-INE-Student', 'Student')
    page = f"""
      <html>
        <head>
          <title>Test Page</title>
        </head>
        <body>
          <p>{greet} {who}!</p>
        </body>
      </html>
    """
    resp = make_response(page)
    resp.mimetype = 'text/html'
    resp.headers['X-INE-Course'] = "HTTP using Python"
    return resp
```

This typical of a simple server.  A few objects are imported from the `flask` package itself.  We utilize the `request` object to find features of the request made, such as its query parameters or the header fields.  Based on those, we make some decisions about what to put in the response; in the body, in this case and most.  Both `.args` and `.headers` are dictionary like objects that support indexing by key and the `.get()` method.

The function `make_response()` is used to create a response object other than a plain Python string.  This is needed when we want to modify headers.  The `.mimetype` attribute controls the first component of the `Content-Type` header.  Using `.headers` directly lets us insert a custom header.

## Simplest possible

In an absolutely minimal case, the only object you need is `Flask` which is instantiated to create the `app` object that is used by various servers, including by the development server.  An even simpler web application could be:

```python
from flask import Flask
app = Flask(__name__)

@app.route('/hello')
def hello():
    return "Hello"
```

We can use this, but it's slightly imperfect, but usable keeping Postel's Law in mind.

In [2]:
import requests
resp = requests.get('http://popbox.kdm.local:2505/hello')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

print()
print(resp.text)

200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 5
Server: Werkzeug/2.0.0 Python/3.8.10
Date: Sun, 13 Jun 2021 01:04:03 GMT

Hello


The only real problem here is that the response claims to have content type `text/html` but it's actually `text/plain`.

### Streaming

Within both `server1.py` and `server2.py` are slightly different implementations of the `/stream` path.  Flask is able to use a generator to incrementally return response data as it becomes available.  Let's look at one version:

```python
from random import random, choice
from time import sleep
from flask import Response

greetings = Path('greetings.txt').read_text().splitlines()

@app.route('/stream')
def streamed_response():
    def generate():
        for _ in range(10):
            greet = choice(greetings)
            yield greet
            yield '\n'
            sleep(random()*3)
    return Response(generate(), mimetype='text/plain')
```

As well as the structure of a generator constructing a `Response` instance, this example illustrates using a global variable that makes the server stateful.  In this case, we only read from the list of greetings; but in principle it is a mutable object that could be modified during processing requests.  In more robust and production-ready servers, we are likely to store data in a database or similar, but globals may certainly be used for mutable state as well.

### Special responses

One special response type we saw was a redirect.  The status code defaults to 302, but for our purposes we wished to use a `301 Moved Permanently`.

```python
from flask import redirect

@app.route('/redirect')
def to_kdm():
    return redirect("http://kdm.training", code=301)
```

Another special function is best to use with 4xx or 5xx status codes.  For example:

```python
from flask import abort

@app.route('/notfound')
def notfound():
    return abort(404)
```

In [3]:
resp = requests.get('http://popbox.kdm.local:2505/notfound')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

print()
print(resp.text)

404 NOT FOUND
Content-Type: text/html; charset=utf-8
Content-Length: 232
Server: Werkzeug/2.0.0 Python/3.8.10
Date: Sun, 13 Jun 2021 01:11:42 GMT

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In a practical application, when the route is used you will presumably perform conditional checks to determine whether `abort()` or returning another response is appropriate.

### Utilizing forms

Commonly on both web pages and with automated services, it is useful to include structured data within a request.  It is also common to return JSON content as a structured response.  Browsers can process data within their embedded JavaScript, but other clients can equally process JSON as a useful data format.

```python
from flask import jsonify

@app.route('/json', methods=["GET", "POST", "PUT"])
def json():
    if request.method in {"PUT", "POST"}: 
        resp = dict(request.form)
    else:
        resp = dict(request.args)
    resp['Server'] = "Test Server"
    return jsonify(resp)
```

In this example, the same route will look at any form data from a PUT or POST request, but will look at query parameters from a GET request.  In whichever case, it will return a JSON representation of that structured request.  Obviously, real world responses will return derived data instead, but using JSON format will still be relevant.

In [4]:
resp = requests.get('http://popbox.kdm.local:2505/json?name=David&title=Instructor')
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

print()
print(resp.text)

200 OK
Content-Type: application/json
Content-Length: 61
Server: Werkzeug/2.0.0 Python/3.8.10
Date: Sun, 13 Jun 2021 01:15:56 GMT

{"Server":"Test Server","name":"David","title":"Instructor"}



If we process the body of PUT/POST requests instead we get the same kind of response, which we probably want to treat as Python native data.

In [5]:
import json

data = {'name': 'David', 'title': 'Data Scientist'}
resp = requests.post('http://popbox.kdm.local:2505/json', data=data)
print(resp.status_code, resp.reason)
for k, v in resp.headers.items():
    print(f'{k}: {v}')

print()
print(json.loads(resp.text))

200 OK
Content-Type: application/json
Content-Length: 65
Server: Werkzeug/2.0.0 Python/3.8.10
Date: Sun, 13 Jun 2021 01:17:33 GMT

{'Server': 'Test Server', 'name': 'David', 'title': 'Data Scientist'}


### Authorization

We utilized Basic Authentication in an earlier lesson.  Let's see how that was implemented by the server.  This requires having installed a Flask extension called Flask-HTTPAuth.  The module contains other authentication methods as well, but HTTPBasicAuth is easy to illustrate.

```python
from flask_httpauth import HTTPBasicAuth
from werkzeug.security import generate_password_hash, check_password_hash

auth = HTTPBasicAuth()

users = {
    "David": generate_password_hash("4bYaDZCFsTY4"),
}

@auth.verify_password
def verify_password(username, password):
    if username in users and \
            check_password_hash(users.get(username), password):
        return username

@app.route('/secure')
@auth.login_required
def secure():
    return f"Hello, {auth.current_user()}!"
```

In [6]:
from requests.auth import HTTPBasicAuth
resp = requests.get('http://popbox.kdm.local:2502/secure', 
                    auth=HTTPBasicAuth('David', '4bYaDZCFsTY4'))
resp, resp.text

(<Response [200]>, 'Hello, David!')

In [7]:
from requests.auth import HTTPBasicAuth
resp = requests.get('http://popbox.kdm.local:2502/secure', 
                    auth=HTTPBasicAuth('David', 'badpass'))
resp, resp.text

(<Response [401]>, 'Unauthorized Access')